In [ ]:
%maven org.dflib:dflib-jupyter:2.0.0-SNAPSHOT

In [ ]:
import org.dflib.parquet.*;
import org.dflib.echarts.*;

## Top Points

In [ ]:
var top_ss = Parquet.loader()
    .cols("season_id", "game_type_id", "name", "points", "goals", "assists")
    .load("_data/skater_season.parquet")
    .rows(and( $int("game_type_id").eq(2) , $int("season_id").eq(20232024) )).colsExcept("season_id", "game_type_id").select()
    .sort($col("points").desc())
    .head(20);

top_ss

In [ ]:
var p1 = ECharts.chart()
    .grid(Grid.of().bottomPct(35))
    .legend(Legend.ofPlain().unselected("assists"))
    .renderAsSvg()
    .tooltip(Tooltip.ofAxis())
    .toolbox(Toolbox.of().featureSaveAsImage().featureDataZoom().featureRestore())
    .sizePx(1000, 400)
    .xAxis("name", XAxis.ofCategory().label(
        AxisLabel.of().rotate(90).fontSize(9)
    ))
    .series(SeriesOpts.ofBar().stack().itemStyle(
            BarItemStyle.of().opacity(0.5).borderRadius(5)
        ), "goals", "assists")
    .series(SeriesOpts.ofLine().showSymbol(false).lineStyle(LineStyle.of().width(1)), "points")
    .plot(top_ss);

p1

## Weight by Year

In [ ]:
var w_skater = Parquet.loader()
    .cols("season_id", "game_type_id", "player_id", "name", "position", "weight")
    .load("_data/skater_season.parquet")
    .rows(and( $int("game_type_id").eq(2), $int("weight").gt(0) ))
          .colsExcept("game_type_id").select()
    .group("player_id")
        .sort($col("season_id").asc()).head(1)
        .cols("season_id", "name", "position", "weight")
        .select()
    .cols("year", "position", "weight", "name")
        .select(
            $int("season_id").div(10_000),
            $col("position"),
            $col("weight"),
            $col("name")
        );

w_skater

In [ ]:
var w_goalie = Parquet.loader()
    .cols("season_id", "game_type_id", "player_id", "name", "position", "weight")
    .load("_data/goalie_season.parquet")
    .rows(and( $int("game_type_id").eq(2), $int("weight").gt(0) ))
          .colsExcept("game_type_id").select()
    .group("player_id")
        .sort($col("season_id").asc()).head(1)
        .cols("season_id", "name", "position", "weight")
        .select()
    .cols("year", "position", "weight", "name")
        .select(
            $int("season_id").div(10_000),
            $col("position"),
            $col("weight"),
            $col("name")
        );

w_goalie

In [ ]:
var w = w_skater
    .vConcat(w_goalie)
    .cols("Offense", "Defense", "Goaltenders")
        .merge(
            ifExp($col("position").in("R", "L", "C"), $col("weight"), $val(null)),
            ifExp($col("position").eq("D"), $col("weight"), $val(null)),
            ifExp($col("position").eq("G"), $col("weight"), $val(null))
        )
    .sort($col("year").asc(), $col("position").asc(), $col("name").asc());

w

In [ ]:
java.util.function.Function<String, ScatterSeriesOpts> seriesMaker = color -> 
    SeriesOpts.ofScatter().symbolSize(6).itemStyle(ScatterItemStyle.of().color(color));

var p2 = ECharts.chart()
    .legend()
    .renderAsSvg()
    .tooltip(Tooltip.ofItem())
    .toolbox(Toolbox.of().featureSaveAsImage().featureDataZoom().featureRestore())
    .sizePx(1000, 400)
    .xAxis("year")
    .series(seriesMaker.apply("black"), "Offense")
    .series(seriesMaker.apply("teal"), "Defense")
    .series(seriesMaker.apply("yellow"), "Goaltenders")
    .plot(w);

p2

In [ ]:
ECharts.saver()
    .createMissingDirs()
    .htmlTemplate("hockey_players.mustache")
    .save("../_out/hockey_players.html", p1, p2) 

In [ ]:
ECharts.saver()
    .createMissingDirs()
    .htmlTemplate("hockey_players_site.mustache")
    .save("../_out/_site/hockey_players.html", p1, p2) 